![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
OutputParser
</b></font> </br></p>


---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>Umgebung einrichten</font> </br></p>
!uv pip install --system --prerelease allow -q git+https://github.com/ralf-42/genai_lib
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

# 1 | OutputParser
---

LangChain stellt eine Vielzahl von Ausgabeparsern bereit, die speziell darauf ausgelegt sind, Informationen aus den Ergebnissen großer Sprachmodelle (LLMs) effizient zu extrahieren und zu strukturieren. Diese Parser sind essenzielle Bestandteile der LangChain-Architektur und häufig zentrale Elemente sogenannter LangChain-Ketten. Solche Ketten bestehen aus konfigurierbaren Abfolgen von Operationen, die Modellausgaben verarbeiten und für weiterführende Anwendungen nutzbar machen.


**Warum sind OutputParser so wichtig?**
+ LLMs geben standardmäßig unstrukturierte Texte zurück.
+ OutputParser sind nötig, um das LLM-Output strukturiert weiterzuverarbeiten.
+ Besonders bei komplexen Anwendungen (z. B. Ketten mit mehreren Modellen, Agenten oder RAG-Systemen) müssen die Antworten klar definiert sein.

# 2 | Strukturierter Parser (einfach)
---


Der **Structured Output Parser** in LangChain ermöglicht die strukturierte Verarbeitung der Ausgaben großer Sprachmodelle (LLMs). Dies ist besonders hilfreich, wenn Informationen aus mehreren Feldern extrahiert und kategorisiert werden müssen. Durch die Segmentierung der Modellausgabe in definierte Bereiche wird die Interpretation und Handhabung der Daten vereinfacht.

Während der **Pydantic/JSON-Parser** eine leistungsfähigere Lösung für komplexe Datenstrukturen darstellt, eignet sich der **Structured Output Parser** besonders für Umgebungen mit begrenzten Rechenressourcen oder für Modelle mit geringer Leistungsfähigkeit. Er bietet eine einfache und effiziente Möglichkeit, die Ausgabe zu strukturieren, ohne das System zu stark zu belasten.

Der erste Schritt bei der Nutzung dieses Parsers besteht in der Erstellung eines **ResponseSchemas**, das definiert, welche Werte extrahiert werden sollen. Jeder Wert wird dabei detailliert beschrieben. Anschließend wird der **StructuredOutputParser** aus einer Liste dieser Schemata erstellt.

In [ ]:
# Abschnitt 0: Install
!uv pip install --system --prerelease allow -q langchain-core langchain_community langchain_openai gradio

In [ ]:
# Abschnitt 1: Importe
from IPython.display import display, Markdown

# Neue strukturierte Imports für aktuelle LangChain-Versionen
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

<p><font color='blue' size="4">
Ohne Nachbearbeitung mit einem Parser
</font></p>

In [ ]:
model_name = "gpt-4o-mini"
temperature = 0.0


llm = ChatOpenAI(model=model_name, temperature=temperature)

In [ ]:
input = "Was ist Machine Learning?"

response = llm.invoke(input)

In [ ]:
for r in response:
    print(r)

<p><font color='blue' size="4">
Mit Nachbearbeitung mit einem Parser
</font></p>

In [ ]:
parser = StrOutputParser()

chain = llm | parser

response = chain.invoke(input)

In [ ]:
response

# 3 | Strukturieter Parser (mittel)

<p><font color='blue' size="4">
Structured Output Parser (mittel)
</font></p>

**Schritt 1:** Der **Parser** definiert ein Schema für die erwartete Ausgabe und verarbeitet die Antwort des LLMs in ein strukturiertes Format (z.B. ein Python-Dictionary).

In [ ]:
# 1. Parser-Schema definieren
response_schema = [
    ResponseSchema(name="input", description="Frage des Benutzers"),
    ResponseSchema(name="answer", description="Antwort auf die Frage des Benutzers"),
    ResponseSchema(name="source", description="Verwendete Quelle (Website) für die Antwort")
]

# 2. Parser erstellen
parser = StructuredOutputParser.from_response_schemas(response_schema)

# 3. Formatierungs Instruktion zu diesem Parser für das Prompt erstellen
format_instructions = parser.get_format_instructions()

Wir können die einzelnen Elemente anzeigen, die wir zum Abrufen von Daten verwenden werden.

In [ ]:
print(response_schema)

In [ ]:
print(parser)

In [ ]:
print(format_instructions)

**Schritt 2:** Verwendung des Schemas mit der Formatierungs Instruktion in einem Prompt.

In [ ]:
# 1. System-Prompt mit den Variablen (f-format) erstellen
system_prompt = """
    Beantworte die folgende Frage so präzise wie möglich.
    Gib auch die Quelle deiner Information an.

    Frage: {input}

    {format_instructions}
"""

# 2. Prompt-Template generieren
prompt = PromptTemplate(
    template=system_prompt,
    input_variables=["input"],
    partial_variables={"format_instructions": format_instructions}
)

Wir können die Eingabeaufforderung anzeigen, die wir zum Abrufen von Daten verwenden werden.

In [ ]:
print(prompt)

In [ ]:
# Funktion zur Aufbereitung der Druck-Ausgabe
def markdown_out(response):
    mprint(f"### 🧑 Mensch")
    mprint(f"{response['input']}")
    mprint(f"### 🤖 KI:")
    mprint(f"{response['answer']}")
    mprint(f"### Quelle:\n {response['source']}")

<p><font color='black' size="5">
Einbindung des Parser in unterschiedliche Chain-Strukturen.
</font></p>

**Variante 0:** Ohne Chain-Struktur

In [ ]:
# Aufruf
response = llm.invoke(prompt.format(input=input))

# Antwort parsen
parsed_response = parser.parse(response.content)

markdown_out(parsed_response)

**Variante 2:** Chain mit direkte Eingabe als Dictionary

In [ ]:
chain = (
    prompt
    | llm
    | parser
)

# Aufruf mit Dictionary
response = chain.invoke({"input": "Was ist Machine Learning?"})
markdown_out(response)

**Variante 3:** Chain mit Dictionary-Transformation `RunnablePassthrough`

In [ ]:
# Chain mit Dictionary-Transformation
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"input": RunnablePassthrough()}  # Wandelt String in Dictionary um
    | prompt
    | llm
    | parser
)

# Aufruf mit einfachem String
response = chain.invoke(input)
markdown_out(response)

# 4 | Strukturierter Parser (komplex)
---

Nun wird ein Beispiel mit einer größeren Anzahl an Werten getestet. Das folgende Programm nimmt Texte in beliebigen Sprachen entgegen und übersetzt sie ins Englische, Spanische und Chinesische.

**Schritt 1:** Der **Parser** definiert ein Schema für die erwartete Ausgabe und verarbeitet die Antwort des LLMs in ein strukturiertes Format (z.B. ein Python-Dictionary).

In [ ]:
response_schema = [
    ResponseSchema(name="detected", description="Die Sprache ist Nutzer-Eingabe"),
    ResponseSchema(name="german", description="German translation"),
    ResponseSchema(name="english", description="English translation"),
    ResponseSchema(name="spanish", description="Spanish translation"),
    ResponseSchema(name="chinese", description="Chinese translation"),
]
parser = StructuredOutputParser.from_response_schemas(response_schema)

format_instructions = parser.get_format_instructions()

**Schritt 2:** Verwendung des Schemas mit der Formatierungs Instruktion in einem Prompt.

In [ ]:
# 1. System-Prompt mit den Variablen (f-format) erstellen
system_prompt = """
    Übersetze in die angegebenen Sprachen.\n{format_instructions}\n{input}
"""

# 2. Prompt-Template generieren
prompt = PromptTemplate(
    template=system_prompt,
    input_variables=["input"],
    partial_variables={"format_instructions": format_instructions}
)

**Variante 2:** Chain mit direkte Eingabe als *Dictionary*

Zunächst wird ein deutscher Satz getestet, um zu beobachten, wie die Übersetzung in die drei Zielsprachen erfolgt.

In [ ]:
chain = prompt | llm | parser
input = "Wann wurde Python eingeführt?"
chain.invoke({"input": input})

Dann wird ein französischer Satz getestet, um zu beobachten, wie die Übersetzung in die drei Zielsprachen erfolgt.

In [ ]:
chain = prompt | llm | parser
question = "Who rides so late through night and wind?"
chain.invoke({"input": input})

# 5 | Datenformat Parser
---

LangChain bietet eine breite Palette an Parsern, die unterschiedliche Datenformate verarbeiten können, was seine Einsatzmöglichkeiten erheblich erweitert. Es unterstützt unter anderem die nahtlose Integration von Pandas-Datenrahmen, kommagetrennten Listen, JSON-Strukturen sowie Datums- und Zeitobjekten. Diese Flexibilität ermöglicht eine effiziente Anpassung an verschiedene Arten von Dateneingaben und macht LangChain zu einem leistungsstarken Werkzeug für die Analyse und Verarbeitung von Daten. Im Folgenden werden einige dieser Parser genauer betrachtet, ihre praktischen Anwendungen demonstriert und aufgezeigt, wie sie zur Optimierung von Prozessen und zur Gewinnung wertvoller Erkenntnisse beitragen können.

<p><font color='black' size="5">
JSON
</font></p>

Die Ausgabe des LLM kann in einem JSON-Format strukturiert werden.

In [ ]:
#
# Variante: Prompt
#
from langchain_openai import ChatOpenAI

prompt = (
    "Bitte gib mir eine Person mit Name und Alter im JSON-Format zurück, z.B. "
    '{"name": "Max", "age": 30}'
)

model_name = "gpt-4o-mini"
temperature = 0.0

llm = ChatOpenAI(model=model_name, temperature=temperature)

response = llm.invoke(prompt)

mprint("### 🤖 KI:")
mprint("---")
mprint(response.content)

In [ ]:
#
# Variante: JsonOutputParser
#
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# Parser
parser = JsonOutputParser()

# Prompt
prompt = ChatPromptTemplate.from_template(
    "Give name and age of a person in JSON format."
)

# Kette
chain = prompt | llm | parser

# Ausführen
response = chain.invoke({})

mprint("### 🤖 KI:")
mprint("---")
print(response)

In diesem Beispiel wird ein Satz erkannt, der als Englisch identifiziert wird, und anschließend ins Spanische, Französische und Chinesische übersetzt.

In [ ]:
# Abschnitt 1: Importe
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Abschnitt 2: Funktionen
def translate_text(text: str) -> dict:
    """Übersetzt den Text in mehrere Sprachen"""
    # Parser ohne Pydantic-Modell konfigurieren
    parser = JsonOutputParser()
    llm = ChatOpenAI(model="gpt-4", temperature=0.0)

    # Prompt erstellen mit direkten Anweisungen zum JSON-Format
    prompt_template = """
    Übersetze die Benutzereingabe in mehrere Sprachen und gib das Ergebnis als JSON zurück.

    Dein JSON sollte folgende Felder enthalten:
    - detected: Die erkannte Ausgangssprache
    - german: Die deutsche Übersetzung
    - spanish: Die spanische Übersetzung
    - french: Die französische Übersetzung
    - chinese: Die chinesische Übersetzung

    Benutzereingabe: {input}
    """

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["input"]
    )

    # Kette ausführen
    chain = prompt | llm | parser
    return chain.invoke({"input": text})

# Abschnitt 3: Hauptprogramm
# Beispieltext übersetzen und Ergebnisse ausgeben
result = translate_text("What is your name?")

print()
mprint(f"**Erkannte Sprache:** {result['detected']}")
mprint(f"**Deutsch:** {result['german']}")
mprint(f"**Spanisch:** {result['spanish']}")
mprint(f"**Französisch:** {result['french']}")
mprint(f"**Chinesisch:** {result['chinese']}")

<p><font color='black' size="5">
Datum/Uhrzeit
</font></p>

Langchain bietet mit dem **DatetimeOutputParser** eine spezielle Funktion zur Interpretation von Datums- und Zeitangaben aus Texten. Sie erkennt unterschiedliche Formate und wandelt diese in ein standardisiertes Zeitformat um. Dies ist besonders hilfreich für Anwendungen wie Terminplanung, Datenanalyse oder andere Bereiche, in denen eine präzise Verarbeitung zeitlicher Informationen erforderlich ist. Der **DatetimeOutputParser** erleichtert die Verarbeitung von Datums- und Uhrzeitangaben und trägt dazu bei, dass Anwendungen zeitbezogene Daten effizient verwalten und nutzen können.

In [ ]:
# Abschnitt 1: Importe
from langchain.output_parsers import DatetimeOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

In [ ]:
template = """Beantworte die Frage zu genau wie möglich.
Wenn Du die Frage nach dem Datum nicht beantworten kannst, dann geben mir das Datum 01.01.1111.

{input}

{format_instructions}"""

# Abschnitt 3: Komponenten initialisieren
# model_name = "gpt-4o-mini"
# temperature = 0.0

# llm = ChatOpenAI(model="gpt-4o-mini",
#     temperature=0.2,
#     n=1
# )

parser = DatetimeOutputParser()

prompt = PromptTemplate(
    template=template,
    input_variables=['input'],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)

Wir können die Eingabeaufforderung anzeigen, die wir zum Abrufen von Daten verwenden werden.

In [ ]:
print(prompt)

Wir erstellen die Kette, die wir zum Parsen von Daten verwenden werden.

In [ ]:
chain = prompt | llm | parser

Wir werden nach zwei Daten fragen, einem realen und einem fiktiven.

In [ ]:
response = chain.invoke({"input": "Wann wurde die Progammiersprache Python eingeführt?"}) # 20.02.1991
print(response)

In [ ]:
response = chain.invoke({"input": "An welchem Tag startet der Krieg im Videospiel Fallout?"}) #  23.10.2077
print(response)

# 6 | Eigene Parser
---

In bestimmten Szenarien kann es sinnvoll sein, einen benutzerdefinierten Parser zu erstellen, um die Modellausgabe eindeutig zu formatieren.  

Dafür bietet sich die Verwendung von **RunnableLambda** oder **RunnableGenerator** in LCEL an, was für viele Fälle ein guten Ansatz ist.  

<p><font color='black' size="5">
Anwendungsfall
</font></p>



Large Language Models (LLMs) wie GPT-4 können Text generieren, der Code und erklärende Beschreibungen nahtlos vermischt. Dies kann zwar für Lern- und Dokumentationszwecke unglaublich nützlich sein, kann aber eine Herausforderung darstellen, wenn aus solchen Ausgaben mit gemischtem Inhalt nur der Code extrahiert und ausgeführt werden muss. Um dies zu beheben, implementieren wir eine einfache Funktion, die nicht-Python-Codezeilen aus einer gegebenen Textzeichenfolge entfernt.

Bei diesem Ansatz werden reguläre Ausdrücke verwendet, um Zeilen zu identifizieren und beizubehalten, die der typischen Python-Syntax entsprechen, während Zeilen verworfen werden, die beschreibender Text zu sein scheinen. Aufgrund der inhärenten Komplexität und Variabilität sowohl von Python-Code als auch von natürlicher Sprache kann diese Methode jedoch nie perfekt sein. Sie basiert auf heuristischen Mustern, die Code manchmal fälschlicherweise als Text klassifizieren oder umgekehrt.

Im nächsten Abschnitt werden wir untersuchen, wie ein anderes LLM beim Entfernen von Nicht-Python-Code helfen kann und möglicherweise eine ausgefeiltere und genauere Lösung bietet. Das folgende Beispiel enthält eine Mischung aus LLM-Kommentaren und generiertem Code.









In [ ]:
from langchain_core.runnables import RunnableLambda

# Benutzer-Funktion zur Trennung von Erläuterung und Code
def parse_ai_message(ai_message):
    """Trennt die Erläuterung und den Code aus einer AIMessage und gibt sie separat zurück."""
    text = ai_message.content  # Extrahiere den reinen Textinhalt der AIMessage

    if "```" in text:
        # Trennen der Erläuterung und des Codes
        parts = text.split("```")
        explanation = parts[0].strip()
        code = parts[1].strip() if len(parts) > 1 else ""
    else:
        # Falls kein Codeblock vorhanden ist, geben wir nur die Erläuterung zurück
        explanation = text.strip()
        code = ""

    return explanation, code  # Rückgabe von zwei separaten Werten

# 3. RunnableLambda für das Parsing erstellen
parser = RunnableLambda(parse_ai_message)

# 4. LLM und Parser verketten
chain = llm | parser

# 5. Eingabe an die Pipeline senden und Ergebnis ausgeben
prompt = """
    Erkläre mir, wie man eine einfache Funktion in Python erstellt und gib ein Beispiel zur Berechnung von Primzahlen.
    Verwende bei Formeln das Format $ Formel $
"""
explanation, code = chain.invoke(prompt)

mprint("### 🤖 Erläuterung")
mprint(explanation)
mprint("### 🤖 Code")
print(code)

# A | Aufgabe
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.


<p><font color='black' size="5">
JSON-Parser mit LangChain
</font></p>


**Ziel:** Verständnis für den Einsatz von `JsonOutputParser` in LangChain.

**Aufgabe:**  
1. Nutze den `JsonOutputParser` von LangChain, um eine KI-Antwort in JSON zu formatieren.  
2. Lasse ein Language Model (z. B. OpenAI GPT) eine Liste von drei zufälligen Städten in Deutschland generieren.  
3. Verwende den Parser, um die Ausgabe in ein JSON-Format umzuwandeln.

**Erwartete Ausgabe (Beispiel):**
```json
{
  "cities": ["Berlin", "Hamburg", "München"]
}
```



<p><font color='black' size="5">
Extraktion von Schlüsselwerten
</font></p>


**Ziel:** Nutzung des `PydanticOutputParser`, um strukturierte Daten aus natürlicher Sprache zu extrahieren.

**Aufgabe:**  
1. Definiere eine `Pydantic`-Datenklasse mit den Feldern: `name` (str), `alter` (int), `stadt` (str).  
2. Verwende ein Language Model, um aus einer gegebenen Beschreibung eine Person zu extrahieren.  
3. Parse die Antwort mit dem `PydanticOutputParser`.

**Beispiel-Eingabe:**
> "Max Mustermann ist 35 Jahre alt und lebt in Berlin."

**Erwartete Ausgabe (Pydantic-Modell):**
```json
{
  "name": "Max Mustermann",
  "alter": 35,
  "stadt": "Berlin"
}
```


<p><font color='black' size="5">
Parser für Listenformate
</font></p>


**Ziel:** Implementierung eines eigenen Parsers zur Umwandlung von KI-Ausgaben in Listen.

**Aufgabe:**  
1. Erstelle eine eigene Parser-Klasse, die eine durch Kommas getrennte Liste in ein Listenformat umwandelt.  
2. Verwende diesen Parser, um eine Liste von fünf beliebten Büchern aus einer Language-Model-Antwort zu extrahieren.

**Beispiel-Eingabe:**
> "Die Verwandlung, Faust, Der Prozess, Die Blechtrommel, Der Vorleser"

**Erwartete Ausgabe:**
```python
["Die Verwandlung", "Faust", "Der Prozess", "Die Blechtrommel", "Der Vorleser"]
```

<p><font color='black' size="5">
Kombination Parser & PromptTemplate
</font></p>


**Ziel:** Verwendung von `StructuredOutputParser` in Kombination mit `PromptTemplate`.

**Aufgabe:**  
1. Erstelle ein `PromptTemplate`, das eine strukturierte Antwort über ein Land ausgibt (Name, Hauptstadt, Einwohnerzahl).  
2. Verwende den `StructuredOutputParser`, um die Antwort in ein Dictionary zu konvertieren.

**Beispiel-Prompt:**
> "Gib mir Informationen zu Frankreich im folgenden JSON-Format: { 'name': '...', 'hauptstadt': '...', 'einwohner': ... }."

**Erwartete Ausgabe:**
```json
{
  "name": "Frankreich",
  "hauptstadt": "Paris",
  "einwohner": 67000000
}
```
